In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import Dash
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mongodb_crud import AnimalShelter

# Initialize the AnimalShelter instance
db = AnimalShelter()

# Fetch initial data from the database
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True, errors='ignore')

# Dashboard Layout / View

app = Dash(__name__)
image_filename = 'test.png' 

# Encode the header image
if os.path.exists(image_filename):
    with open(image_filename, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode()
else:
    encoded_image = ''

app.layout = html.Div([
    html.Center([
        html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height':'10%', 'width':'10%'}) if encoded_image else None,
        html.B(html.H1('CS-340 Dashboard by Eric Buchanan'))
    ]),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='filter-options',
            options=[
                {'label': 'Water Rescue', 'value': 'OPT1'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'OPT2'},
                {'label': 'Disaster or Individual Tracking', 'value': 'OPT3'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',  # default selected value
            style={'display': 'inline-block', 'margin-right': '10px'}  # added style
        ),
    ], id='interactive-buttons', style={'display': 'flex', 'justify-content': 'center'}),  # added style
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        # Enable sorting
        sort_action="native",
        # Enable filtering
        filter_action="native",
        # Allow single row to be selected
        row_selectable="single",
        # Pagination settings
        page_action="native",
        page_current=0,
        page_size=10,
        # Allow cells to be edited
        editable=True,
        style_table={'overflowX': 'auto'},  # Enable horizontal scrolling
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display' : 'flex', 'flex-wrap': 'wrap'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                     style={'flex': '1', 'minWidth': '300px', 'padding': '10px'}
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                     style={'flex': '1', 'minWidth': '300px', 'padding': '10px'}
                 )
             ])
])

# Mapping from the radio button values to the preferred breeds
radio_button_to_breeds_mapping = {
    'OPT1': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'],
    'OPT2': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
    'OPT3': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'],
}

# Mapping from the radio button values to age ranges
radio_button_to_age_upon_outcome_mapping = {
    'OPT1': {'$gte': 26, '$lte': 156},  # 26 weeks to 156 weeks
    'OPT2': {'$gte': 26, '$lte': 156},  # 26 weeks to 156 weeks 
    'OPT3': {'$gte': 20, '$lte': 300},  # 20 weeks to 300 weeks
}

# Mapping from the radio button values to sex upon outcome
radio_button_to_sex_upon_outcome_mapping = {
    'OPT1': ['Intact Female'],
    'OPT2': ['Intact Male'],
    'OPT3': ['Intact Male'],
}

@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('filter-options', 'value')]
)
def update_dashboard(filter_type):
    # Updates the data table based on the selected filter option
    try:
        # Check which button has been pressed and set the query
        if filter_type == 'reset':
            query = {}  # Reset button pressed, so no filter   
        else:
            # Map filter_type to the preferred breeds
            preferred_breeds = radio_button_to_breeds_mapping.get(filter_type, [])
            age_range = radio_button_to_age_upon_outcome_mapping.get(filter_type, {})
            preferred_sex = radio_button_to_sex_upon_outcome_mapping.get(filter_type, [])
            query = {
                'breed': {'$in': preferred_breeds},
                'age_upon_outcome_in_weeks': age_range,
                'sex_upon_outcome' : {'$in': preferred_sex}
            }

        # Run the database query
        records = db.read(query)

        # Convert records to DataFrame
        df_filtered = pd.DataFrame(records)

        if not df_filtered.empty and '_id' in df_filtered.columns:
            df_filtered['_id'] = df_filtered['_id'].apply(str)

        # Prepare the data for the DataTable
        data = df_filtered.to_dict('records')
        columns = [{"name": i, "id": i} for i in df_filtered.columns if i != '_id']
        
        return data, columns
    except Exception as e:
        print(f"Error updating dashboard: {e}")
        return [], []

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    # Updates the pie chart based on the current data in the data table
    if not viewData or not any(viewData):
        # Create an empty pie chart to avoid the warning
        empty_df = pd.DataFrame({'breed': pd.Series(dtype='object'), 'count': pd.Series(dtype='int')})
        fig = px.pie(empty_df, values='count', names='breed', title='Preferred Animals')
        return dcc.Graph(figure=fig)
    
    dff = pd.DataFrame.from_dict(viewData)
    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']
    
    # Limit the number of breeds displayed on the pie chart
    top_breeds = breed_counts.nlargest(10, 'count')  # Top 10 breeds
    other_breeds_sum = breed_counts['count'][10:].sum()  # Sum of other breeds
    
    # Append the "Other" category using pandas.concat
    other_breeds_df = pd.DataFrame({'breed': ['Other'], 'count': [other_breeds_sum]})
    top_breeds = pd.concat([top_breeds, other_breeds_df], ignore_index=True)
    
    fig = px.pie(top_breeds, values='count', names='breed', title='Preferred Animals')
    
    return dcc.Graph(figure=fig)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # Highlights the selected columns in the data table
    if selected_columns is None:
        # If selected_columns is None, return an empty list to avoid TypeError
        return []

    # Return a list of dictionaries that define the conditional styling
    return [
        {
            'if': {'column_id': column},
            'background_color': '#D2F3FF'
        } for column in selected_columns
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, derived_virtual_selected_rows):
    # Updates the geolocation map based on the selected row in the data table
    # Default map that gets returned if there is no data or no selection
    default_map = dl.Map(style={'width': '100%', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
        dl.TileLayer()
    ])

    # If there is no data or no row is selected, return the default map
    if not viewData or not derived_virtual_selected_rows:
        return default_map

    # Convert viewData to DataFrame and get the selected row
    dff = pd.DataFrame.from_dict(viewData)
    selected_index = derived_virtual_selected_rows[0]
    if selected_index >= len(dff):
        return default_map

    selected_row = dff.iloc[selected_index]
    latitude = selected_row.get('location_lat')
    longitude = selected_row.get('location_long')

    # Validate latitude and longitude
    if pd.isnull(latitude) or pd.isnull(longitude):
        return default_map

    try:
        latitude = float(latitude)
        longitude = float(longitude)
    except ValueError:
        return default_map

    # Create the map with a marker for the selected animal location
    return dl.Map(style={'width': '100%', 'height': '500px'}, center=[latitude, longitude], zoom=12, children=[
        dl.TileLayer(),
        dl.Marker(position=[latitude, longitude])
    ])

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)
